In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer

from genomic_benchmarks.dataset_getters.pytorch_datasets import DemoHumanNontataPromoters
from utils import coll_factory, LetterTokenizer, build_vocab
from cnn_model import CNN

## Choose the dataset

In [4]:
get_dataset_fn = DemoHumanNontataPromoters
train_dset = get_dataset_fn('train', force_download=False, version=0)

Reference /home/jupyter/.genomic_benchmarks/fasta/Homo_sapiens.GRCh38.dna.toplevel.fa.gz already exists. Skipping.


  0%|          | 0/24 [00:00<?, ?it/s]

## Tokenizer and vocab

In [6]:
tokenizer = get_tokenizer(LetterTokenizer())
vocabulary = build_vocab(train_dset, tokenizer, use_padding=False)
vocabulary.append_token('N')


print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 7
{'N': 6, 'T': 3, '<eos>': 5, 'G': 2, 'C': 4, 'A': 1, '<bos>': 0}


## Batch preparation with collate

In [7]:
RUN_ON_GPU = True
device = 'cuda' if RUN_ON_GPU and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

collate = coll_factory(vocabulary, tokenizer, device)
train_loader = DataLoader(train_dset, batch_size=32, shuffle=True, collate_fn=collate)

Using cuda device


## Model

In [8]:
max_seq_len = max([len(train_dset[i][0]) for i in range(len(train_dset))])
print("max_seq_len ", max_seq_len)
# Count in added in tokenizer '<bos>' and '<eos>' 
nn_input_len = max_seq_len+2

model = CNN(
    number_of_classes=1,
    vocab_size=vocabulary.__len__(),
    embedding_dim=100,
    input_len=nn_input_len
).to(device)

max_seq_len  251


/opt/conda/envs/bench_env/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


## Training

In [9]:
model.train(train_loader, epochs=10)

Epoch 0
Train metrics: 
 Accuracy: 80.0%, Avg loss: 0.616821 

Epoch 1
Train metrics: 
 Accuracy: 82.9%, Avg loss: 0.606362 

Epoch 2
Train metrics: 
 Accuracy: 83.4%, Avg loss: 0.603001 

Epoch 3
Train metrics: 
 Accuracy: 83.0%, Avg loss: 0.599323 

Epoch 4
Train metrics: 
 Accuracy: 84.2%, Avg loss: 0.595568 

Epoch 5
Train metrics: 
 Accuracy: 81.1%, Avg loss: 0.600004 

Epoch 6
Train metrics: 
 Accuracy: 85.0%, Avg loss: 0.593839 

Epoch 7
Train metrics: 
 Accuracy: 85.1%, Avg loss: 0.593579 

Epoch 8
Train metrics: 
 Accuracy: 85.7%, Avg loss: 0.590632 

Epoch 9
Train metrics: 
 Accuracy: 85.9%, Avg loss: 0.588893 



## Testing

In [10]:
test_dset = get_dataset_fn('test', force_download=False, version=0)
test_loader = DataLoader(test_dset, batch_size=32, shuffle=True, collate_fn=collate)

model.test(test_loader)

Reference /home/jupyter/.genomic_benchmarks/fasta/Homo_sapiens.GRCh38.dna.toplevel.fa.gz already exists. Skipping.


  0%|          | 0/24 [00:00<?, ?it/s]

test_loss  168.8089716732502
num_batches 283
correct 7632
size 9034
Test Error: 
 Accuracy: 84.5%, Avg loss: 0.596498 

